# Соединения Солнца и Астрономических Объектов

## 1. Немного о вычислениях

В астрономии используется несколько видов координат. Один из них - геоцентрические; представьте поверхность Земли с нанесённой на неё разметкой в виде меридианов и параллелей, при этом последние удаляются в бесконечность, образуя т.н. небесную сферу. С точки зрения земного наблюдателя по небесной сфере движутся все астрономические объекты. Измеряя их положение в одно и то же фикисированное время суток на протяжении времени, примерно соответствующему одному земному году, можем заметить, что астрономические объекты 

Теперь вспомним, что Земля вращается вокруг Солнца под углом. 

### Вторая экваториальная система:

Основная плоскость - плоскость небесного экватора, в случае геоцентрической координатной системы совпадает с плоскостью Земного экватора.

Координаты второй экваториальной системы - (прямое восхождение, склонение) = (Ra, Dec).

$\textbf{Ra}$ (обозначается как $\alpha$) - англ. $\textbf{right ascension}$ - прямое восхождение, дуга небесного экватора от точки весеннего равноденствия до круга склонения светила, или угол между направлением на точку весеннего равноденствия и плоскостью круга склонения светила. Является эквивалентом долготы, отсчитывается от точки весеннего равноденствия (напомним, так называют точку, где Солнце пересекает небесный экватор в весеннее время года).

$\textbf{Dec}$ (обозначается как $\delta$) - англ. $\textbf{declination}$ - склонение, равняется угловому расстоянию на небесной сфере от плоскости небесного экватора до объекта; имеет знак ($+ \delta$ для объектов выше небесного экватора (север), $- \delta$ для объектов ниже (расположенных к югу). Является эквивалентом земной широты, отсчитывается от плоскости небесного экватора (в этой же плоскости лежит земной экватор).

Обе величины могут быть записаны в одном из двух форматов: в часовом $\textbf{h:m:s}$, где $h \in [0,24)$, $m, s \in [0, 60)$ (часы:минуты:секунды), либо угловом $\bf{a^o \, b' \, c''}$ (или $\textbf{a:b:c}$), $ a \in [0, 360)$ для $\alpha$ и $a \in [-90, 90]$ для $\delta$, $b,c \in [0, 60)$ (градусы:минуты:секунды).

### Приближение времени соединения Солнца и астрономического объекта

Считаем, что объект априори образует соединение с Солнцем (т.е., окзывается в некоторой малой окрестности Солнца с точки зрения наблюдателя). Для определения времени нам потребуются Ra ($\alpha$) координата Солнца и объекта. Обозначим их как $\alpha_{Sun}$ и $\alpha_{obj}$.

Здесь важно сделать ремарку о том, в каком формате доступны координаты Солнца. Таблицы, содержащие эти координаты для заданного объекта (а также и ряд других параметров) называют $\textbf{эфемеридами}$. Эфемериды вычисляются с учётом локального положения наблюдателя на поверхности Земли. Данный скрипт использует эфемериды Александровича, доступные по ссылке: http://hea.iki.rssi.ru/~nik/ak/sun.htm . Указанные эфемериды Солнца вычислены на каждый день для часового пояса UTC +03:00 (12:00 по Московскому зимнему времени). На момент написания данного текста эфемериды доступны с 1.12.2014 по 31.01.2016.

Таким образом, в нашем распоряжении оказываются координаты Солнца, вычисленные с шагом в одни сутки. Для приближённого вычисления времени соединения можно вычислить среднюю скорость движения Солнца по Ra-координате за 24 часа:
$$v_{\text{ср}} = \frac{\alpha_{Sun}^{[1]} - \alpha_{Sun}^{[0]}}{\Delta t} = \frac{\Delta \alpha_{Sun}}{\Delta t},$$
где $\Delta t = 24$ часа, $\alpha_{Sun}^{[0]}, \, \alpha_{Sun}^{[1]}$ - соседние измерения Ra-координаты Солнца.

Далее, из предположения о том, что соединения в указанный 24-часовой промежуток (от полудня первого дня до полудня дня второго) всё-таки произойдёт, вытекает, что в некоторый момент времени Ra-координаты Солнца и объекта совпадут. Что же, тогда не составляет никакого труда вычислить длину пути по $\alpha$-координате:
$$s = \alpha_{obj} - \alpha_{Sun}^{[0]},$$
и время соединения (в приближении равномерного движения по Ra-координате!) вычисляется по простой школьной формуле путём деления длины пути на скорость движения:
$$t_{appr.} = t_0 + \frac{s}{v_{\text{ср}}},$$
где индекс "appr." является сокращением от "approximate" (приближённый), а $t_0$ - время измерения $\alpha_{Sun}^{[0]}$.

### Вычисление удалённости положения объекта от Солнечной эклиптики

Здесь тоже всё относительно просто: необходимы $\delta$-координаты Солнца в момент соединения и объекта, из координаты Солнца вычитаем координату объекта, получаем расстояние, на котором объект отстоит от Солнца по данной координате:
$$\rho(\text{Sun}, \text{obj}) = \delta_{\text{Sun}} - \delta_{\text{obj}},$$
т.к. $\alpha$-координаты в момент соединения совпадают. Координаты $\delta_{\text{obj}}$ для астрономических объектов доступны из астрономических каталогов. С Dec-координатой для Слнца приходится немного сложнее.


Существуют формулы для приближённого вычисления склонения Солнца в зависимости от дня в году и времени суток. Здесь была использована следующая формула:
$$\delta \approx \frac{\pi}{180} \arcsin\left[ \sin\left(-23.44^o \frac{\pi}{180}\right) \cos\left(\frac{2 \pi}{365.24}(N+10) + 2 \cdot 0.0167 \cdot \sin\left(\frac{2 \pi}{365.24} (N-2)\right)\right) \right]$$

Параметр N изменяется от 0 до 365(366), при этом полагается изменяющимся непрерывно. Это даёт возможность учитывать суточное время. Так, для 12 часов 1.01 числа $N=0.5$.




## График функции приближённого склонения Солнца

[Nmin, Nmax] - отрезок числа дней, для которых будет построен график.

step - шаг по времени. По умолчанию выбрали в один час [step = 1 / 24]

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

Nmin = 0
Nmax = 365*2
step = 1. / 24.

def sundec(N):
    tmp = np.sin(-23.44 * np.pi / 180)
    tmp *= np.cos(2*np.pi / 365.24 * ( N + 10) + 2*0.0167*np.sin(2*np.pi / 365.24*(N-2)))
    return np.arcsin(tmp)*180/np.pi

N = np.arange(Nmin, Nmax, step)
f = sundec(N)

plt.plot(N, f)
plt.xlabel('Day')
plt.ylabel('Delta')

## 2. Каталоги астрономических объектов

### Сервисы

Существует большое количество самых разных астрономических каталогов. Наиболее привычное для человека представление - табличное, однако часто это уже конечная форма представления данных. Для хранения чаще всего используют базы данных. Здесь приведём несколько популярных сервисов, где можно обращаться с запросами к самым разным каталогам (сохранённым в виде баз данных):

1) Vizier
http://vizier.u-strasbg.fr/viz-bin/VizieR

Представлено более 14000 каталогов. Принцип работы прост: сначала необходимо выбрать (найти) интересующий каталог, далее вести работу внутри выбранного каталога.

2) Simbad
http://simbad.u-strasbg.fr/simbad/

Данный сервис, наоборот, ориентирован на объекты. Использует информацию из многих источников; удобен для поиска информации по отдельно взятому объекту или объектам вне зависимости от баз данных. 

3) Minor Planets Center (MPC)
http://www.minorplanetcenter.com/

Здесь можно найти информацию о всех малых объектах вроде астероидов или комет.

### Ссылки


1) Hipparcos Input Catalogue (HIP / HIC)

https://en.wikipedia.org/wiki/Hipparcos
Звёздный каталог.

2) Обозначения Флемстида (F-код)

https://ru.wikipedia.org/wiki/Обозначения_Флемстида

3)  Обозначения Байера (B-код)

https://ru.wikipedia.org/wiki/Обозначения_Байера

4) Новый общий каталог туманностей и звёздных скоплений (NGC)

https://en.wikipedia.org/wiki/New_General_Catalogue

5) Abell catalogue

https://en.wikipedia.org/wiki/Abell_catalogue

6) Messier object

https://en.wikipedia.org/wiki/Messier_object

7) Henry Draper Catalogue

https://en.wikipedia.org/wiki/Henry_Draper_Catalogue


### Каталоги

V50

V239A

In [ ]:
#N,i,w,a,e,M - orbital elements
#http://www.stjarnhimlen.se/comp/ppcomp.html